In [10]:
import torch
from torchvision import models, transforms
from PIL import Image
import json
model = models.vgg16(pretrained=True)
model.eval() 

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),        # Resize to 224x224 as required by VGG16
    transforms.ToTensor(),                # Convert the image to a PyTorch tensor
    transforms.Normalize(                 # Normalize using ImageNet's mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])
image_path = '../datasets/images/im7.jpg'
image = Image.open(image_path)
image = preprocess(image)
image = image.unsqueeze(0)  

with torch.no_grad():      
    outputs = model(image)
with open('labels.json') as f:
    labels = json.load(f)
_, indices = torch.sort(outputs, descending=True)
top5_indices = indices[0][:5].tolist()
top5_labels = [labels[idx] for idx in top5_indices]
top5_probs = torch.softmax(outputs, dim=1)[0, top5_indices].tolist()
for i, (label, prob) in enumerate(zip(top5_labels, top5_probs)):
    print(f"{i+1}: {label} ({prob * 100:.2f}%)")

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1: tiger cat (51.46%)
2: Egyptian Mau (18.46%)
3: lynx (13.75%)
4: tabby cat (13.48%)
5: bow tie (0.47%)
